In [ ]:
# Imports, always include this at the beginning

# No science without NumPy
import numpy as np

# Matplotlib for plotting
import matplotlib.pyplot as plt
%matplotlib notebook

# Of course we want Glotaran
import glotaran as gta
# This things we just need for simulations, you can leave them out for fitting
from lmfit import Parameters

In [ ]:
# First we define our model specification. See the documentation for details how to do so.
model_spec = '''
type: kinetic

parameters: [101e-1, 202e-5,505e-6]

compartments: [s1, s2, s3]

megacomplexes:
    - label: mc1
      k_matrices: [k1]

k_matrices:
  - label: "k1"
    matrix: {
      '("s1","s1")': 1,
      '("s2","s2")': 2,
      '("s3","s3")': 3,
}

initial_concentrations: []

irf: []

shapes:
  - label: "shape1"
    type: "gaussian"
    amplitude: shape.1
    location: shape.2
    width: shape.3
  - ["shape3", "gaussian", shape.6, shape.7, shape.8]

datasets: 
  - label: dataset1
    type: spectral
    megacomplexes: [mc1]
'''

# Then we can create our model
model = gta.parse(model_spec)

In [ ]:
# Now let us define some simulation parameters
sim_params = Parameters()
sim_params.add("p_1", 101e-3)
sim_params.add("p_2",202e-4)
sim_params.add("p_3",505e-5)

# We also need a spectral, a time axis and some amplitudes
spectral_axis = np.arange(600, 750, 1)
time_axis = np.arange(0, 1500, 1.5)
amplitudes = [1,0.1,7]

# Now we can simulate the data
model.simulate('dataset1', {"spectral": spectral_axis, "time": time_axis}, parameter=sim_params)

In [ ]:
# Plot the simulated data

data = model.datasets["dataset1"].data.data

fig = plt.figure()
plt.xlabel('Time (ps)')
plt.ylabel('Pixel')
plt.pcolormesh(time_axis, spectral_axis, data)
plt.show()

In [ ]:
result = model.fit()

In [ ]:
result.best_fit_parameter.pretty_print()

In [ ]:
lsv, s, rsv = result.final_residual_svd()

In [ ]:
fig, [ax1, ax2, ax3] = plt.subplots(1, 3)
fig.set_size_inches(10, 3)
ax1.set_title('lsv')
ax1.pcolormesh(lsv)
ax2.set_title('s')
ax2.plot(s[:5])
ax3.set_title('rsv')
ax3.pcolormesh(rsv)

In [ ]:
plt.pcolormesh(result.eval('dataset1'))